описание

In [92]:
import json
import pandas as pd
from sklearn.cluster import DBSCAN
import ctypes
from welly import Well, Project, Curve
import matplotlib.pyplot as plt
import os

with open('famdict.json', 'r', encoding='utf-8') as f:
    famdict = json.load(f)
with open('lqcdict.json', 'r', encoding='utf-8') as f:
    lqcdict = json.load(f)

In [ ]:
#словари перенести в json файлы
famdict = {'CALI': ["Caliper", "mm"],
           'GR': ["Gamma Ray", "Gapi"],
           'RHOB': ["Bulk Density", "g/cm3"],
           'NPHI': ["Neutron Porosity", "v/v"],
           'PEF': ["Photoelectric Factor", "b/e"],
           'IL': ["Induction Resistivity", "Ohmm"],
           'IL1': ["Induction Resistivity", "Ohmm"],
           'IL2': ["Induction Resistivity", "Ohmm"],
           'IL3': ["Induction Resistivity", "Ohmm"],
           'IL4': ["Induction Resistivity", "Ohmm"],
           'IL5': ["Induction Resistivity", "Ohmm"],
           'RTIL': ["Formation Resistivity", "Ohmm"],
           'LL': ["Lateral Log Resistivity", "Ohmm"],
           'LL1': ["Lateral Log Resistivity", "Ohmm"],
           'LL2': ["Lateral Log Resistivity", "Ohmm"],
           'LL3': ["Lateral Log Resistivity", "Ohmm"],
           'LL4': ["Lateral Log Resistivity", "Ohmm"],
           'LL5': ["Lateral Log Resistivity", "Ohmm"],
           'RTLL': ["Formation Resistivity", "Ohmm"],
           'RT': ["Formation Resistivity", "Ohmm"],
           'SP': ["Spontaneous Potential", "mV"],
           'DTP': ["Compressional Slowness", "us/m"],
           'DTS': ["Shear Slowness", "us/m"],
           'MPHI': ["NMR Porosity", "v/v"],
           'NKTB': ["Neutron Far", "UE"],
           'MGZ': ["Micro Gradient Resistivity", "Ohmm"],
           'MPZ': ["Micro Potential Resistivity", "Ohmm"]}

lqcdict = {'CALI': ['DS', 'CALI', 'ДС'],
           'GR': ['GK', 'ГК', 'GR'],
           'RHOB': ['GGK', 'RHOB', 'ГГКп', 'GGKP'],
           'NPHI': ['WS', 'TRNP', 'W'],
           'PEF': ['PEF'],
           'IL': ['ИК', 'IK'],
           'IL1': ['RO05', 'IVR1', 'IK05'],
           'IL2': ['RO07', 'IVR2', 'IK07'],
           'IL3': ['RO10', 'IVR3'],
           'IL4': ['RO14', 'IVR4', 'IK12'],
           'IL5': ['RO20', 'IVR5', 'IK20'],
           'RTIL': ['RT_IK', 'RPZ'],
           'LL': ['BK', 'БК'],
           'LL1': ['L1FC', 'BK1'],
           'LL2': ['L2FC', 'BK2'],
           'LL3': ['L3FC', 'BK3'],
           'LL4': ['L4FC', 'BK4'],
           'LL5': ['L5FC', 'BK5'],
           'RTLL': ['RT_BK', 'REK_BK'],
           'RT': ['ROP', 'RT'],
           'SP': ['PS', 'SP'],
           'DTP': ['DTp', 'DTP', 'DT', 'DTCO'],
           'DTS': ['DTs', 'DTs', 'DTSM', 'DTS'],
           'MPHI': ['TCMR'],
           'NKTB': ['CFTC', 'НКТб', 'NKTB'],
           'MGZ': ['MGZ'],
           'MPZ': ['MPZ'],
           'GZ1': ['A04M01N', 'GZ1'],
           'GZ2': ['A1M01N', 'GZ2'],
           'GZ3': ['A2M05N', 'GZ3'],
           'GZ4': ['A4M05N', 'GZ4'],
           'GZ5': ['A8M1N', 'GZ5'],
           'PZ': ['N6M05A', 'PZ'],
           'MLL': ['MBK', 'BMK'], }

#write dicts in json with cyrilic
#with open('famdict.json', 'w', encoding='utf-8') as f:
#	json.dump(famdict, f, ensure_ascii=False)
#with open('lqcdict.json', 'w', encoding='utf-8') as f:
#	json.dump(lqcdict, f, ensure_ascii=False)

#может всетаки реформатить словарь чтоб ключем была мнемоника кривой и ее lqc имя. В ловать такде записать фемилис и юнит. Словарь будет больше но один и быстрее.


In [93]:
def datasetlist_sort(dbdatasetlist):
    dsets = []
    distance_typing = 300  #параметр для настройки типизации интервалов записи
    for dset in dbdatasetlist:
        top = dset[1]
        bottom = dset[2]
        if top > bottom:
            top, bottom = bottom, top
        dsets.append([dset[0], top, bottom, bottom - top])
    ds_df = pd.DataFrame(dsets, columns=['project_index', 'top', 'bottom', 'size'])
    ds_df.sort_values(by=['top', 'size'], inplace=True)

    dbs = DBSCAN(eps=distance_typing, min_samples=1).fit(ds_df[['top', 'bottom']])
    ds_df['labels'] = dbs.labels_

    print(ds_df.to_string())

    return ds_df[['project_index', 'labels']].values.tolist()



In [96]:

def lqclogdata(dataset_ind, dslabel_):
    logs = []
    lqcname = ''
    for cname in las_project[dataset_ind].data.keys():
        if any(cname in sublist for sublist in list(lqcdict.values())):  # есть ли кривая в словаре lqcdict гделибо
            lqcname = list(lqcdict.keys())[list(lqcdict.values()).index(
                [sublist for sublist in list(lqcdict.values()) if cname in sublist][
                    0])] + dslabel_  #выбираем из словаря ключ соотвествующий найденному в значениях имени кривой
            if lqcname in wLQC.data.keys():
                clist = [i for i in wLQC.data.keys() if i.find(lqcname) != -1]
                if len(clist) >= 1:
                    lqcname = lqcname + "_" + str(len(clist))

            logs.append([cname, lqcname])

        #if db.variableUnit(well, ds, lqcname)!=famdict[lqcname[0:version_index]][1]:
        # 	crunit = ""
        #
        # 	if lqcname[0:version_index] == "NPHI": #добавить проверку по конкретным методам ГК НК
        # 		if  float(db.variableInformation(well, ds, lqcname," Mean"))<1:
        # 			crunit = "v/v"
        # 		else:
        # 			crunit = "%"
        #
        # 	elif lqcname[0:version_index] == "GR" :
        # 		if float(db.variableInformation(well, ds, lqcname, "Mean"))<50:
        # 			crunit = "uR/H"
        # 		else:
        # 			crunit = "Gapi"
        #
        # 	elif lqcname[0:version_index] == "CALI" :
        # 		if float(db.variableInformation(well, ds, lqcname, "Mean"))<1:
        # 			crunit = "m"
        # 		else:
        # 			crunit = "mm"
        # 	else:
        # 		crunit = famdict[lqcname[0:version_index]][1]
    return logs

In [99]:
#указать папку с ласами они могут быть из разных скважин

las_path = "/Users/victorpotysyev/PycharmProjects/lqc_maker/"

#find list of las files in folder las_path including subfolders
las_list = []
for root, dirs, files in os.walk(las_path):
    for file in files:
        if file.lower().endswith(".las"):
            las_list.append(os.path.join(root, file))


las_project = Project.from_las(las_list, index='M')

wells_list = {}
for ind, w in enumerate(las_project):

    well_name = str(w.name).replace(" ", "")
    ds_start = w.header[w.header['mnemonic'] == 'STRT']['value'].values[0]
    ds_stop = w.header[w.header['mnemonic'] == 'STOP']['value'].values[0]

    if well_name == 'WELL' or well_name == '':
        well_name = str(w.fname)
        print("!!! file {} does not contain well name. Using file name instead".format(w.fname)) #задавать вручную

    if well_name not in wells_list.keys() :
        wells_list[well_name] = []

    wells_list[well_name].append([ind, ds_start, ds_stop])
print(wells_list)







0it [03:03, ?it/s]
0it [02:39, ?it/s]
0it [02:49, ?it/s]
0it [00:53, ?it/s]
/opt/anaconda3/lib/python3.9/site-packages/welly/las.py:144: UserWarning: Warning, LAS version 3.0 not yet supported. Attempting to use LAS 1.2 and 2.0 parsing logic for LAS 3.0.
  datasets = from_lasio(las)





1it [00:00,  3.02it/s]/opt/anaconda3/lib/python3.9/site-packages/welly/las.py:144: UserWarning: Warning, LAS version 3.0 not yet supported. Attempting to use LAS 1.2 and 2.0 parsing logic for LAS 3.0.
  datasets = from_lasio(las)





2it [00:00,  4.37it/s]/opt/anaconda3/lib/python3.9/site-packages/welly/las.py:144: UserWarning: Warning, LAS version 3.0 not yet supported. Attempting to use LAS 1.2 and 2.0 parsing logic for LAS 3.0.
  datasets = from_lasio(las)





3it [00:00,  3.45it/s]/opt/anaconda3/lib/python3.9/site-packages/welly/las.py:144: UserWarning: Warning, LAS version 3.0 not yet supported. Attempting to use LAS 1.2 and 2.0 parsing logic for LAS 3.0.
  datasets = from_lasio(las)





4

!!! file /Users/victorpotysyev/PycharmProjects/lqc_maker/Mikhalevskoye_1P/D3/MKV_D3.LAS does not contain well name. Using file name instead
{'1607': [[0, 2124, 3455], [1, 2025, 3447.8], [2, 2025, 3456.1], [3, 2025, 3462.6]], '511R': [[4, 3188.1, 3519.1], [5, 3203.3, 3520.3], [6, 1157.04, 3191.34], [7, 112.9, 1250.3], [8, 1142.74, 3180.54], [9, 3160.6, 3520.3], [10, 1004.82, 3201.42], [11, 3285.6, 3517.4], [12, 567.26, 1252.66], [13, 1153.57, 3191.27], [14, 3159.91, 3528.51], [15, 1156.6, 3190.9], [16, 3186.1, 3524.5], [17, 565.5, 1282.05], [18, 1183.4, 3198], [19, 3159.91, 3528.51], [20, 523.1, 1257.4], [21, 516.2, 1240.3]], '1П': [[22, 1197.0, 1439.4], [23, 1197.0, 1439.4], [24, 1197.0, 1439.4], [25, 1781.0, 1988.5], [26, 1781.0, 1988.5], [27, 1781.0, 1988.5], [28, 1781.0, 1988.5], [29, 1781.0, 1988.5], [30, 1781.0, 1988.5], [31, 1781.0, 1988.5], [32, 1781.0, 1988.5], [33, 1781.0, 1988.5], [34, 1781.0, 1988.5], [35, 1781.0, 1988.5], [36, 1781.0, 1988.5], [37, 1781.0, 1988.5], [38, 229

In [137]:
for well in wells_list:
    print(well)
    #determine maximum depth for lqc dataset and create list for dataframe index

    max_depth = max([w[2] for w in wells_list[well]])
    min_depth = min([w[1] for w in wells_list[well]])
    #MD = [round(d * 0.1, 1) for d in range(0, int(round((max_depth + 5) / 0.1, 0)) + 1)]

    wLQC = Well()
    wLQC.name = well



    for dataset_ind, dslabel in datasetlist_sort(wells_list[well]):
        if dslabel == 0:
            dslabel_ = ""
        else:
            dslabel_ = "_" + str(dslabel)

        #how can i change mnemonic of curve object
        #copy curve object correctly to preserve unit information
        #assign or check units for curves
        for cname, lqcname in lqclogdata(dataset_ind, dslabel_):

            wLQC.data[lqcname] = Curve(data=las_project[dataset_ind].data[cname].values, index=las_project[dataset_ind].data[cname].index,
                      mnemonic=lqcname ).to_basis(start=min_depth-5, stop=max_depth + 5, step=0.1)

    if len(wLQC.data.keys()) > 0:
        wLQC.to_las(well +"_LQC.las")

   project_index   top  bottom    size  labels
1              1  2025  3447.8  1422.8       0
2              2  2025  3456.1  1431.1       0
3              3  2025  3462.6  1437.6       0
0              0  2124  3455.0  1331.0       0
    project_index      top   bottom     size  labels
3               7   112.90  1250.30  1137.40       0
17             21   516.20  1240.30   724.10       1
16             20   523.10  1257.40   734.30       1
13             17   565.50  1282.05   716.55       1
8              12   567.26  1252.66   685.40       1
6              10  1004.82  3201.42  2196.60       2
4               8  1142.74  3180.54  2037.80       2
9              13  1153.57  3191.27  2037.70       2
11             15  1156.60  3190.90  2034.30       2
2               6  1157.04  3191.34  2034.30       2
14             18  1183.40  3198.00  2014.60       2
10             14  3159.91  3528.51   368.60       3
15             19  3159.91  3528.51   368.60       3
5               9  3160